# STITCH API

Full documentation available at http://stitch.embl.de/help/api/

<img src="http://stitch.embl.de/help/images/api_format.png" width="650px" align="left" alt="API diagram" >

In [1]:
from urllib2 import Request, urlopen, URLError
import json

### Demo

All high-confidence (interaction score ≥ 700) human protein targets of amiodarone

In [2]:
# Input drug
drug = 'amiodarone'

In [3]:
# Get molecule identifier
request = Request('http://stitch.embl.de/api/json/resolve?identifier=%s&species=9606' %drug)

try:
    response = urlopen(request).read()
    decoded = json.loads(response)
    stitch_id = int(decoded[0]['stringId'].split('CID')[1])
    print drug, stitch_id
except URLError, e:
    print '%s not found in STITCH' %drug, e

amiodarone 100002156


In [4]:
# Get interacting proteins (can't return as JSON, must be psi-mi (XML) or psi-mi-tab (TSV-like))
min_score = 700 # range [0-1000]
request = Request('http://stitch.embl.de/api/psi-mi-tab/interactionsList?identifiers=-%d&species=9606&required_score=%d'
                 %(stitch_id, min_score) )
try:
    response = urlopen(request).read()
    for line in response.split('\n'):
        row = line.split('\t')
        if len(row) > 1 and 'CID' in row[1]: # check for empty rows and that interaction is with a drug
            if int(row[1].split('CID')[1]) != stitch_id: # confirm correct stitch_id for drug
                continue
            protein_id = row[0].split('.')[1]
            gene_name = row[2]
            score = row[14].split('|')[0].split(':')[1]
            print protein_id, gene_name, score#, row
        
except URLError, e:
    print 'Proteins interacting with %s not found in STITCH' %drug, e

ENSP00000337915 CYP3A4 0.985
ENSP00000237612 ABCG2 0.956
ENSP00000353820 CYP2D6 0.915
ENSP00000427336 D3 0.9
ENSP00000262186 KCNH2 0.88
ENSP00000260682 CYP2C9 0.871
ENSP00000355627 AGT 0.849
ENSP00000275216 TAAR1 0.826
ENSP00000342007 CYP1A2 0.823
ENSP00000265724 ABCB1 0.82
